<a href="https://colab.research.google.com/github/ubettercallsaul/Project-ML1/blob/main/5_Pyomo_Linear_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Programming

Linear programming (LP) is a mathematical optimization problem which is formulated using linear functions. The general formulation of LP is described as

$$
\begin{align*}
    \min \quad & c^\top x\\
    \text{s.t.} \quad & A x \leq b\\
    & x \geq 0, \nonumber
\end{align*}
$$

where $x$ is a vector of the design variables which has $n$ number of the design variables so that $x \in \mathbb{R}^n$. $c \in \mathbb{R}^n$ are the objective coefficients. Therefore, $c^\top x$ is the cost function. The $m$ linear constraints are described by the matrix $A \in \mathbb{R}^{m \times n}$ and the vector $b \in \mathbb{R}^m$.

This post will not discuss various optimization solvers for the LP, but focus on how to solve LP problems in Python using [Pyomo](http://www.pyomo.org/).

## Battery Production Planning

## The microchip production problem
The company GL  produces two types of Li-ion battery Cathodes, NMC (1 g Lithium, 7 g Nickel, 1 g Manganese, 1 g Cobalt) and NCA (1 g :Lithium, 6 g Nickel, 2 g Cobalt, 1g Aluminium). Each of NMC can be sold for a 9 USD profit, and each of NCA for a 5 USD profit.




 Cathode |Lithium | Nickel | Manganese | Cobalt| Aluminiium| Profit
------------------ |------------------ |------------------ |------------------ |------------------ |------------------ |------------------
NMC | 1g | 7g| 1g | 1g | 0g | 9 USD
NCA | 1g | 6g| 2g | 0g | 1g | 5 USD


Current stock of raw materials: 200 g Lithium, 1200 g Nickel, 150 g Manganese, 200 g Cobalt, 100g Aluminum


Lithium | Nickel | Manganese | Cobalt| Aluminiium
------------------ |------------------ |------------------ |------------------ |------------------
200g | 1200g| 150g | 200g | 100g




**Q: How many Cathodes of each type should be produced to maximize the profit while respecting the raw material stock availability? **

Let $x_1$ denote the number of NMC and $x_2$ that of NCA. This decision can be reformulated as an optimization problem of the following form:


$$
\begin{align}
\max  \quad  & 9 x_1 + 5 x_2 \\
\text{s.t.} \quad
    &   x_1 + x_2 \leq 200 &\text{Lithium}\\
    &   7x_1 + 6x_2 \leq 1200 &\text{Nickel}\\
    &   x_1 \leq 150 &\text{Manganese}\\
    &   x_1 + 2x_2  \leq 200 &\text{Cobalt}\\
    &   x_2 \leq 100 &\text{Aluminum}\\
    &   x_1, x_2 \geq 0
\end{align}
$$

The problem has $n=2$ decision variables and $m=4$ constraints. Using the standard notation introduced above, denote the vector of decision variables by $x = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$ and define the problem coefficients as

$$
\begin{align*}
    c = \begin{pmatrix} 9 \\ 5 \end{pmatrix},
    \qquad
    A =
    \begin{bmatrix}
    1 & 1\\
    7 & 6\\
    1 & 0\\
    1 & 2\\
    0 & 1\\
    \end{bmatrix},
    \quad \text{ and } \quad
    b = \begin{pmatrix} 200 \\ 1200 \\ 150 \\ 200 \\100 \end{pmatrix}.
\end{align*}
$$

This model can be implemented and solved in Pyomo as follows.

In [ ]:
# install pyomo and select solver
import sys

SOLVER = "cbc"

if "google.colab" in sys.modules:
    !pip install highspy >/dev/null
    SOLVER = "appsi_highs"

In [ ]:
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pyomo.environ as pyo

m = pyo.ConcreteModel('Cathode')

m.x1 = pyo.Var(domain=pyo.NonNegativeReals)
m.x2 = pyo.Var(domain=pyo.NonNegativeReals)

m.profit = pyo.Objective(expr=9*m.x1 + 5*m.x2, sense=pyo.maximize)

m.lithium = pyo.Constraint(expr=m.x1 + m.x1 <= 200)
m.nickel = pyo.Constraint(expr=7*m.x1 + 6*m.x2 <= 1200)
m.manganese = pyo.Constraint(expr=m.x1 <= 150)
m.cobalt = pyo.Constraint(expr=1*m.x1 + 2*m.x2 <= 200)
m.aluminum = pyo.Constraint(expr=m.x2 <= 100)

pyo.SolverFactory(SOLVER).solve(m)

print(f'x = ({m.x1.value:.1f}, {m.x2.value:.1f})')
print(f'optimal value = {pyo.value(m.profit):.2f}')



x = (100.0, 50.0)
optimal value = 1150.00


The maximum profit is \$1150 when producing 100 NMC and 50 NCA. 😀

**References**

https://mobook.github.io/MO-book/intro.html